In [3]:
from __future__ import print_function

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [5]:
import torchvision
import torchvision.transforms as transforms

In [6]:
import os
import sys
import time
import argparse
import numpy as np

In [7]:
from torch.autograd import Variable

In [17]:
import config as cf
from dataset import dataset as dset

In [18]:
use_cuda = torch.cuda.is_available()

In [19]:
best_acc = 0

In [25]:
# Data Uplaod
print('\n[Phase 1] : Data Preparation')
trainloader, testloader, outputs, inputs = dset('mnist')
print ('Number of classes: {} \n Number of input channels: {}'.format(outputs,inputs))


[Phase 1] : Data Preparation
| Preparing MNIST dataset...
| Number of classes: 10 
 Number of input channels: 1


| Preparing CIFAR-10 dataset...


In [29]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [15]:

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform(m.weight, gain=np.sqrt(2))
        nn.init.normal_(m.weight, mean=0, std=1)
        nn.init.constant(m.bias, 0)

class AlexNet(nn.Module):

    def __init__(self, num_classes, inputs=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(inputs, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [30]:
net = AlexNet(num_classes = 10,inputs=3)
file_name = 'alexnet-'

In [31]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [32]:
criterion = nn.CrossEntropyLoss()

In [38]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, lr))
    for batch_idx, (inputs_value, targets) in enumerate(trainloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        print ('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainset)//batch_size)+1, loss.data[0], 100.*correct/total))

In [39]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs_value, targets) in enumerate(testloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda()
        with torch.no_grad():
            inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)
        loss = criterion(outputs, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.data[0], acc))
    
    if acc > best_acc:
        best_acc = acc

In [40]:
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

In [22]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.0010


/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


| Epoch [  1/ 10] 		Loss: 2.3043 Acc@1: 7.000%
| Epoch [  1/ 10] 		Loss: 2.3030 Acc@1: 7.000%
| Epoch [  1/ 10] 		Loss: 2.3016 Acc@1: 8.000%
| Epoch [  1/ 10] 		Loss: 2.2975 Acc@1: 8.000%
| Epoch [  1/ 10] 		Loss: 2.2847 Acc@1: 9.000%
| Epoch [  1/ 10] 		Loss: 2.2920 Acc@1: 9.000%
| Epoch [  1/ 10] 		Loss: 2.2829 Acc@1: 9.000%
| Epoch [  1/ 10] 		Loss: 2.2713 Acc@1: 10.000%
| Epoch [  1/ 10] 		Loss: 2.2392 Acc@1: 11.000%
| Epoch [  1/ 10] 		Loss: 2.2213 Acc@1: 11.000%
| Epoch [  1/ 10] 		Loss: 2.1893 Acc@1: 12.000%
| Epoch [  1/ 10] 		Loss: 2.1104 Acc@1: 14.000%
| Epoch [  1/ 10] 		Loss: 2.0971 Acc@1: 14.000%
| Epoch [  1/ 10] 		Loss: 1.9626 Acc@1: 15.000%
| Epoch [  1/ 10] 		Loss: 1.8962 Acc@1: 15.000%
| Epoch [  1/ 10] 		Loss: 1.8060 Acc@1: 17.000%
| Epoch [  1/ 10] 		Loss: 1.7210 Acc@1: 18.000%
| Epoch [  1/ 10] 		Loss: 1.7966 Acc@1: 19.000%
| Epoch [  1/ 10] 		Loss: 1.6712 Acc@1: 20.000%
| Epoch [  1/ 10] 		Loss: 1.8403 Acc@1: 20.000%
| Epoch [  1/ 10] 		Loss: 1.7120 Acc@1: 21.000%

| Epoch [  1/ 10] 		Loss: 0.2023 Acc@1: 75.000%
| Epoch [  1/ 10] 		Loss: 0.1291 Acc@1: 75.000%
| Epoch [  1/ 10] 		Loss: 0.0772 Acc@1: 75.000%
| Epoch [  1/ 10] 		Loss: 0.1750 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1320 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1440 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1370 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.0787 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1512 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1140 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.0659 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1423 Acc@1: 76.000%
| Epoch [  1/ 10] 		Loss: 0.1239 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.2332 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1126 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1716 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1803 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1532 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1314 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1018 Acc@1: 77.000%
| Epoch [  1/ 10] 		Loss: 0.1112 Acc@1: 

/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


...

| Epoch [  7/ 10] 		Loss: 0.0344 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0141 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0174 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0784 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0631 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0433 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0218 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0494 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0517 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0233 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0701 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0576 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0317 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0129 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0491 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0529 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0079 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0256 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0136 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0322 Acc@1: 98.000%
| Epoch [  7/ 10] 		Loss: 0.0163 Acc@1: 

| Epoch [  8/ 10] 		Loss: 0.0264 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0101 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0167 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0132 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0125 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0189 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0153 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0341 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0314 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0082 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0080 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0529 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0342 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0110 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0089 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0330 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0490 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0330 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0613 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0176 Acc@1: 99.000%
| Epoch [  8/ 10] 		Loss: 0.0116 Acc@1: 

| Epoch [  9/ 10] 		Loss: 0.0257 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0328 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0236 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0095 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0201 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0374 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0161 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0075 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0212 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0241 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0348 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0267 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0269 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0105 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0112 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0313 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0316 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0149 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0429 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0279 Acc@1: 99.000%
| Epoch [  9/ 10] 		Loss: 0.0209 Acc@1: 

| Epoch [ 10/ 10] 		Loss: 0.0056 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0577 Acc@1: 98.000%
| Epoch [ 10/ 10] 		Loss: 0.0332 Acc@1: 98.000%
| Epoch [ 10/ 10] 		Loss: 0.0136 Acc@1: 98.000%
| Epoch [ 10/ 10] 		Loss: 0.0362 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0173 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0088 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0175 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0074 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0169 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0458 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0470 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0174 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0262 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0433 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0105 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0099 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0088 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0295 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0261 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0075 Acc@1: 

| Epoch [ 10/ 10] 		Loss: 0.0319 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0231 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0265 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0712 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0540 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0422 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0171 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0143 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0389 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0338 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0586 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0275 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0142 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0252 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0357 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0285 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0144 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0116 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0261 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0297 Acc@1: 99.000%
| Epoch [ 10/ 10] 		Loss: 0.0475 Acc@1: 

In [23]:
print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))


[Phase 4] : Testing model
* Test results : Acc@1 = 99.00%


In [43]:
dataset = 'cifar10'

In [44]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.0010


NameError: name 'targets' is not defined

In [42]:
To use:
    %time
    %prun
    Add progress bar : https://github.com/fastai/fastprogress
            

SyntaxError: invalid syntax (<ipython-input-42-99076244e8d2>, line 1)